In [ ]:
# 1. BFS
class Solution:
    def largestPathValue(self, colors: str, edges: List[List[int]]) -> int:
        # 值传递三要素：
        # 1. 状态语义：dp[node][color]
        # 2. 递推公式：dp[node] = max(dp[prev][colors])
        # 3. 传播方向：先序

        # 0. 构建邻接图
        # 1. 初始化indegree
        n = len(colors)
        graph = [[] for _ in range(n)]
        indegree = [0 for _ in range(n)]
        for [x, y] in edges:
            graph[x].append(y)
            indegree[y] += 1

        # *.构建递归所需的结构，color_count
        color_count = [[0]*26 for _ in range(n)]

        # 2. 构建queue
        queue = [i for i in range(n) if indegree[i] == 0]

        # *. 判断是否成环
        topo = 0

        while queue:
            # 3. 出队
            cur = queue.pop(0)
            topo += 1
            color_count[cur][ord(colors[cur])-ord('a')] += 1
            # 4. 扩展
            for next in graph[cur]:
                color_count[next] = list(
                    map(max, color_count[cur], color_count[next]))
                indegree[next] -= 1
                # 5. 入队
                if indegree[next] == 0:
                    queue.append(next)

        if topo != n:
            return -1
        return max([max(node) for node in color_count])


### DFS-postorder

1. path： 先序\*邻接
2. memo：后序\*邻接
3. 要点 1：path 的 true 和 false 是在做回溯，之间不允许 break、continue、return！！
4. 要点 2：path 的无效判断一定要在 memo 重叠之前，否则 memo 重叠判断没有任何意义，因为自己返回了自己初始的赋值，不被允许！！！


In [ ]:
class Solution:
    def largestPathValue(self, colors: str, edges: List[List[int]]) -> int:
        n = len(colors)
        graph = [[] for _ in range(n)]
        for [x, y] in edges:
            graph[x].append(y)

        def forEachColor(color, n):
            cycle = False
            path = [False]*n
            memo = [-1]*n

            def dfs(x):
                nonlocal cycle
                nonlocal colors
                nonlocal memo
                nonlocal path
                nonlocal color
                nonlocal n

                if path[x]:
                    cycle = True
                    return
                if memo[x] != -1:
                    return

                path[x] = True
                color_index = ord(colors[x])-ord('a')
                if color_index == color:  # is color of Node x same as current calculating color
                    memo[x] = 1
                else:
                    memo[x] = 0

                for node in graph[x]:
                    dfs(node)
                    memo[x] = max(
                        memo[x], (memo[node] + (color_index == color)))

                path[x] = False

            for node in range(n):
                dfs(node)
                if cycle:
                    return -1
            return max(memo)

        rlt = 0
        for color in range(26):
            rlt_per_color = forEachColor(color, n)
            if rlt_per_color == -1:
                return -1
            rlt = max(rlt_per_color, rlt)
        return rlt


1857. Largest Color Value in a Directed Graph
      Hard
      1.9K
      64
      Companies
      There is a directed graph of n colored nodes and m edges. The nodes are numbered from 0 to n - 1.

You are given a string colors where colors[i] is a lowercase English letter representing the color of the ith node in this graph (0-indexed). You are also given a 2D array edges where edges[j] = [aj, bj] indicates that there is a directed edge from node aj to node bj.

A valid path in the graph is a sequence of nodes x1 -> x2 -> x3 -> ... -> xk such that there is a directed edge from xi to xi+1 for every 1 <= i < k. The color value of the path is the number of nodes that are colored the most frequently occurring color along that path.

Return the largest color value of any valid path in the given graph, or -1 if the graph contains a cycle.

Example 1:

Input: colors = "abaca", edges = [[0,1],[0,2],[2,3],[3,4]]
Output: 3
Explanation: The path 0 -> 2 -> 3 -> 4 contains 3 nodes that are colored "a" (red in the above image).
Example 2:

Input: colors = "a", edges = [[0,0]]
Output: -1
Explanation: There is a cycle from 0 to 0.

Constraints:

n == colors.length
m == edges.length
1 <= n <= 105
0 <= m <= 105
colors consists of lowercase English letters.
